In [ ]:
!git clone https://github.com/wentaozhu/AutoShot.git

!git clone https://github.com/soCzech/TransNetV2.git


Cloning into 'AutoShot'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 66 (delta 31), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (66/66), 18.82 MiB | 17.49 MiB/s, done.
Resolving deltas: 100% (31/31), done.
fatal: destination path 'TransNetV2' already exists and is not an empty directory.


In [1]:
!pip3 install --force einops==0.4.1
!pip install -q ffmpeg-python pillow

In [2]:
!python3 /content/drive/MyDrive/AIC24/model/AutoShot/compare_inference_baseline_groundtruth_v2.py

Loading pretrained weights from /content/drive/MyDrive/AIC24/model/ckpt_0_200_0.pth
Current model has 90 parameters, updated parameters 90
Traceback (most recent call last):
  File "/content/drive/MyDrive/AIC24/model/AutoShot/compare_inference_baseline_groundtruth_v2.py", line 54, in <module>
    for fnm in os.listdir(video_dir):
FileNotFoundError: [Errno 2] No such file or directory: '/content/video_download'


In [ ]:
import shutil
shutil.make_archive('/content/outputframes', 'zip', '/content/predicted_frames')

'/content/outputframes.zip'

In [ ]:
import os

_, _, files = next(os.walk("/content/predicted_frames"))
file_count = len(files)
print (file_count)

505


In [ ]:
import cv2
import os
import torch
from transnet_v2 import TransNetV2  # Import the TransNetV2 model class from its module

# Define the directory to save frames
output_dir = 'scene_frames'
os.makedirs(output_dir, exist_ok=True)

# Function to save frames from each scene
def save_frames_from_scenes(video_path, scenes, num_frames_per_scene=3):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Iterate through each scene
    for scene_index, (start_frame, end_frame) in enumerate(scenes):
        # Ensure the frame indices are within the video length
        start_frame = max(0, start_frame)
        end_frame = min(total_frames - 1, end_frame)

        # Compute frames to extract from the current scene
        scene_frames = list(range(start_frame, end_frame + 1))
        step = max(len(scene_frames) // num_frames_per_scene, 1)
        sampled_frames = scene_frames[::step][:num_frames_per_scene]

        # Save frames for this scene
        for idx, frame_number in enumerate(sampled_frames):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            ret, frame = cap.read()
            if ret:
                frame_filename = os.path.join(output_dir, f"scene_{scene_index}_frame_{idx}.jpg")
                cv2.imwrite(frame_filename, frame)
                print(f"Saved frame {frame_number} from scene {scene_index} as {frame_filename}")
            else:
                print(f"Failed to read frame {frame_number} from video")

    # Release the video capture object
    cap.release()

# Load the TransNetV2 model
def load_transnet_v2_model(model_path):
    # Initialize the model
    model = TransNetV2()

    # Load pre-trained weights
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set the model to evaluation mode

    return model

# Example usage
video_path = "L01_V001.mp4"
model_path = "path_to_transnetv2_weights.pth"  # Path to your model weights file

# Load the model
model = load_transnet_v2_model(model_path)

# Predict scenes in the video
_, single_frame_predictions, _ = model.predict_video(video_path)
scenes = model.predictions_to_scenes(single_frame_predictions)

# Save 3 frames from each scene
save_frames_from_scenes(video_path, scenes)


NameError: name 'TransNetV2' is not defined